In [1]:
import numpy as np 
import networkx as nx
def draw_graph(networkx_graph,notebook=True,output_filename='graph.html',show_buttons=True,only_physics_buttons=True):
    """
    This function accepts a networkx graph object,
    converts it to a pyvis network object preserving its node and edge attributes,
    and both returns and saves a dynamic network visualization.
    
    Args:
        networkx_graph: The graph to convert and display
        notebook: Display in Jupyter?
        output_filename: Where to save the converted network
        show_buttons: Show buttons in saved version of network?
        only_physics_buttons: Show only buttons controlling physics of network?
    """
    
    # import
    from pyvis import network as net
    
    # make a pyvis network
    pyvis_graph = net.Network(notebook=notebook)
    
    # for each node and its attributes in the networkx graph
    for node,node_attrs in networkx_graph.nodes(data=True):
        pyvis_graph.add_node(str(node),**node_attrs)
        
    # for each edge and its attributes in the networkx graph
    for source,target,edge_attrs in networkx_graph.edges(data=True):
        # if value/width not specified directly, and weight is specified, set 'value' to 'weight'
        if not 'value' in edge_attrs and not 'width' in edge_attrs and 'weight' in edge_attrs:
            # place at key 'value' the weight of the edge
            edge_attrs['value']=edge_attrs['weight']
        # add the edge
        pyvis_graph.add_edge(str(source),str(target),**edge_attrs)
        
    # turn buttons on
    if show_buttons:
        if only_physics_buttons:
            pyvis_graph.show_buttons(filter_=['physics'])
        else:
            pyvis_graph.show_buttons()
    #net.repulsion(node_distance=500, spring_length=200)
    # return and also save
    return pyvis_graph.show(output_filename)



In [18]:
def create_upper_triangular_binary_matrix(size):
    '''
    Cria uma matrix triangular superior, com diagonal principal igual a 0
    
    args: size = Ordem da matriz desejada
    return: matriz triangular superior de ordem igual a desejada
    '''
    
    matrix = np.random.randint(2, size=(size, size))
    np.fill_diagonal(matrix, 0)  # Diagonal = 0
    upper_triangular_matrix = np.triu(matrix)
    return upper_triangular_matrix

# Mutate the matrix by flipping a randomly chosen element in the upper triangular part (excluding the main diagonal)
def mutate_upper_triangular_binary(matrix):

    '''
    Muta uma matrix triangular superior, trocando um elemento 
    aleatório (fora da diagonal) de 0 para 1, ou vice-versa.

    arg: Matriz triangular superior
    return: Matriz triangular superior com uma mutação
    '''
    size = matrix.shape[0]
    
    # Escolhe um elemento aleatório (tirando a diagonal)
    i = np.random.randint(0, size - 1)
    j = np.random.randint(i + 1, size)  # Assegura que estará acima da diagonal principal
    
    # Flipa o valor do elemento escolhido
    matrix[i, j] = 1 - matrix[i, j]
    
    return matrix


def initialize_population(pop_size, matrix_size):
    '''
    Inicializa a população, e já transforma uma matriz triangular
    superior em uma matriz simétrica somando ela com a sua transposta.
    
    args: (tamanho da população, ordem da matriz)
    
    return: População de matrizes simétricas
    '''
    population = []
    for _ in range(pop_size):
        upper_triangular = create_upper_triangular_binary_matrix(matrix_size)
        combined_matrix = upper_triangular + upper_triangular.T
        population.append((upper_triangular, combined_matrix))
    return population


def fitness(combined_matrix):
    '''
    Função de fitness desejada
    
    arg: Matriz simétrica
    return: Valor numérico da fitness function
    '''
    eigenvals = np.linalg.eigvals(combined_matrix)
    autovalores = sorted(eigenvals.real)
    
    e = combined_matrix.shape[0]
    
    #TIRAR ISSO DEPOIS
    if e % 2 == 0:
        ocupados = e // 2 
    elif e % 2 == 1:
        ocupados = (e // 2) + 1
    #CÁLCULO ENERGIA TOTAL SEGUNDO O MÉTODO DE HUCKEL
    t_total = 0
    for n in range(e//2):
        t_total += 2 * autovalores[n]
    if(e%2==1):
        t_total += autovalores[e//2]
    return t_total

# Tournament selection
def tournament_selection(population, fitnesses, k=3):
    '''
    Seleciona um indivíduo da população utilizando o método de seleção por torneio.

    args: 
    population: Lista de matrizes simétricas.
    fitnesses: Lista de valores de fitness correspondentes à população.
    k: Número de indivíduos a serem selecionados para o torneio (default = 3).
    
    return: O indivíduo da população com maior valor de fitness entre os selecionados para o torneio.
    '''
    
    selected_indices = np.random.choice(len(population), k, replace=False)
    selected = [population[i] for i in selected_indices]
    selected_fitnesses = [fitnesses[i] for i in selected_indices]
    return selected[np.argmin(selected_fitnesses)]

# Crossover function
def crossover(parent1, parent2):
    '''
    Realiza o cruzamento entre dois pais para gerar um filho.

    args:
    parent1: Tupla contendo a matriz triangular superior e a matriz simétrica correspondente do primeiro pai.
    parent2: Tupla contendo a matriz triangular superior e a matriz simétrica correspondente do segundo pai.
    
    return: Tupla contendo a matriz triangular superior e a matriz simétrica correspondente do filho gerado.
    '''
    upper_tri1, _ = parent1
    upper_tri2, _ = parent2
    size = upper_tri1.shape[0]
    crossover_point = np.random.randint(1, size)
    
    child_upper_tri = np.zeros((size, size))
    
    # Copy upper part from parent1 and lower part from parent2
    for i in range(size):
        for j in range(size):
            if j >= crossover_point:
                child_upper_tri[i, j] = upper_tri1[i, j]
            else:
                child_upper_tri[i, j] = upper_tri2[i, j]
    
    child_combined = child_upper_tri + child_upper_tri.T
    return (child_upper_tri, child_combined)

# Genetic Algorithm
def genetic_algorithm(pop_size, matrix_size, generations):
    # Initialize population
    population = initialize_population(pop_size, matrix_size)
    
    for generation in range(generations):
        # Evaluate fitness of the population
        fitnesses = [fitness(combined_matrix) for _, combined_matrix in population]
        
        new_population = []
        
        # Generate new population
        for _ in range(pop_size // 2):
            # Select parents
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)
            
            # Crossover
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)
            
            # Mutate
            child1 = (mutate_upper_triangular_binary(child1[0]), child1[0] + child1[0].T)
            child2 = (mutate_upper_triangular_binary(child2[0]), child2[0] + child2[0].T)
            
            new_population.extend([child1, child2])
        
        # Replace old population with new population
        population = new_population
    
    # Final population fitness evaluation
    fitnesses = [fitness(combined_matrix) for _, combined_matrix in population]
    best_individual = population[np.argmin(fitnesses)]
    
    return best_individual, fitnesses

# Parameters
pop_size = 70
matrix_size = 7
generations = 1000

# Run genetic algorithm
best_solution, fit = genetic_algorithm(pop_size, matrix_size, generations)

print("Best solution found:")
print(best_solution[1])  # Print the combined matrix
print("Fitness of the best solution:", fitness(best_solution[1]))

Best solution found:
[[0. 1. 1. 1. 0. 1. 1.]
 [1. 0. 0. 1. 1. 1. 0.]
 [1. 0. 0. 0. 1. 1. 1.]
 [1. 1. 0. 0. 1. 0. 1.]
 [0. 1. 1. 1. 0. 1. 0.]
 [1. 1. 1. 0. 1. 0. 0.]
 [1. 0. 1. 1. 0. 0. 0.]]
Fitness of the best solution: -11.06919452775108


In [16]:
pop_size = 70
matrix_size = 7
generations = 10

# Run genetic algorithm
best_solution, fit = genetic_algorithm(pop_size, matrix_size, generations)

print("Best solution found:")
print(best_solution[1])  # Print the combined matrix
print("Fitness of the best solution:", fitness(best_solution[1]))

Best solution found:
[[0. 1. 0. 0. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1. 0.]
 [0. 1. 0. 1. 1. 0. 1.]
 [0. 1. 1. 0. 0. 1. 0.]
 [1. 1. 1. 0. 0. 0. 1.]
 [1. 1. 0. 1. 0. 0. 1.]
 [1. 0. 1. 0. 1. 1. 0.]]
Fitness of the best solution: -11.069194527751073
